In [1]:
import numpy as np
import os
import pickle

class State:
    def __init__(self, path):
        self.path = path
        if os.path.isfile(path):
            with open(path, "rb") as f:
                self.dict = pickle.load(f)
        else:
            self.dict = {}
        
    def __getitem__(self, k):
        return self.dict[k]
    
    def __setitem__(self, k, v):
        self.dict[k] = v
        with open(self.path, "wb") as f:
            pickle.dump(self.dict, f)

    def get(self, k):
        return self.dict.get(k)
    
state = State(".state.pkl")
for st in state.dict.keys():
    v = state.dict[st]
    if isinstance(v, dict):
        print("{:40.40} {} {}".format(st, v['name'], v['PESQ'].mean()))

0-amr-lq-dense-full_separate             model_2019-04-10T23:55:30.450413.zip 1.7123625
0-amr-lq-dense-full_whole                model_2019-04-11T02:14:55.245116.zip 1.6608875
0-amr-lq-conv_dense-full_separate        model_2019-04-11T04:37:21.520666.zip 1.9748999999999994
0-amr-lq-conv_dense-full_whole           model_2019-04-11T07:45:04.553332.zip 1.8495624999999998
0-amr-lq-conv_dense_4-full_separate      model_2019-04-11T09:11:29.368621.zip 1.9290625
0-amr-lq-conv_dense_4-full_whole         model_2019-04-11T14:19:02.586472.zip 1.8651624999999998
0-amr-lq-dae-full_separate               model_2019-04-11T17:21:20.211222.zip 1.7942500000000003
0-amr-lq-dae-full_whole                  model_2019-04-12T03:52:07.386353.zip 1.8772125000000002
0-amr-lq-dae_4-full_separate             model_2019-04-12T17:22:22.178988.zip 1.8229125
0-amr-lq-dae_4-full_whole                model_2019-04-13T07:19:54.394461.zip 1.9100375
0-amr-hq-dense-full_separate             model_2019-04-13T18:30:21.050186.z

In [2]:
DATASETPATH = "/home/zantyr/.dataset"

model_path = "./runs/2019-04-17T00:50:08.300275.zip"

test_source = np.load(os.path.join(DATASETPATH, "test_source.bin.npy"))
test_target = np.load(os.path.join(DATASETPATH, "test_target.bin.npy"))
test_phase  = np.load(os.path.join(DATASETPATH, "test_phase.bin.npy"))

In [3]:
import sear_lib

model = sear_lib.ModelInstance.load(model_path)

Using TensorFlow backend.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [4]:
spec = np.sqrt(np.exp(-1 * model.model.predict(test_source[:1]))) * np.exp(1j * test_phase[:1])

In [5]:
def istft(spec, size=512, hop=128):
    # plus minus
    time_len = size + hop * spec.shape[0]
    sound = np.zeros(time_len, np.float32)
    for i in range(spec.shape[0] - 1):
        sound[hop * i : hop * i + size] += np.sqrt(np.hamming(size)) * np.fft.irfft(spec[i])
    return sound

sound = istft(spec[0])

In [6]:
import matplotlib.pyplot as plt
plt.plot(sound)
plt.show()

In [7]:
import IPython.display as disp

disp.display(disp.Audio(sound, rate=16000))

In [8]:
spec2 = np.sqrt(np.exp(-1 * test_source[:1])) * np.exp(1j * test_phase[:1, :, :129])
sound2 = istft(spec2[0], size=256, hop=64)
disp.display(disp.Audio(sound2, rate=8000))

In [15]:
import scipy.io.wavfile as sio
import sear_lib
import tqdm

try: os.mkdir("recordings")
except: pass

DATASETPATH = "/home/zantyr/.dataset"

for codec in ["amr-lq", "amr-hq", "gsm-fr"]:
    config = sear_lib.DatasetPreparation.setting_for(codec, 0, training_data=0)
    config.run()
    test_source = np.load(os.path.join(DATASETPATH, "test_source.bin.npy"))
    test_target = np.load(os.path.join(DATASETPATH, "test_target.bin.npy"))
    test_phase  = np.load(os.path.join(DATASETPATH, "test_phase.bin.npy"))
    for rec_ix in tqdm.tqdm(range(test_source.shape[0])):
        spec_pred = np.sqrt(np.exp(-1 * model.model.predict(test_source[rec_ix : rec_ix + 1]))) * np.exp(1j * test_phase[rec_ix : rec_ix + 1])
        sound_pred = istft(spec[0])
        spec_src = np.sqrt(np.exp(-1 * test_source[rec_ix : rec_ix + 1])) * np.exp(1j * test_phase[rec_ix : rec_ix+1, :, :129])
        sound_src = istft(spec2[0], size=256, hop=64)
        spec_src *= 2**15
        spec_src = spec_src.astype(np.int16)
        sound_src *= 2**15
        sound_src = spec_src.astype(np.int16)
        sio.write(os.path.join("recordings", f"{codec}-{rec_ix}_src.wav"), 8000, sound_src)
        sio.write(os.path.join("recordings", f"{codec}-{rec_ix}_pred.wav"), 16000, sound_pred)
    

/home/zantyr/DAE-libri/4021.wav
/home/zantyr/DAE-libri/10645.wav
/home/zantyr/DAE-libri/17795.wav
/home/zantyr/DAE-libri/5101.wav
/home/zantyr/DAE-libri/7254.wav
/home/zantyr/DAE-libri/1646.wav
/home/zantyr/DAE-libri/8494.wav
/home/zantyr/DAE-libri/12788.wav
/home/zantyr/DAE-libri/26681.wav
/home/zantyr/DAE-libri/16156.wav
/home/zantyr/DAE-libri/16905.wav
/home/zantyr/DAE-libri/2922.wav
/home/zantyr/DAE-libri/15643.wav
/home/zantyr/DAE-libri/24998.wav
/home/zantyr/DAE-libri/24176.wav
/home/zantyr/DAE-libri/10967.wav
/home/zantyr/DAE-libri/16585.wav
/home/zantyr/DAE-libri/28436.wav
/home/zantyr/DAE-libri/12373.wav
/home/zantyr/DAE-libri/17676.wav
/home/zantyr/DAE-libri/17476.wav
/home/zantyr/DAE-libri/19791.wav
/home/zantyr/DAE-libri/21401.wav
/home/zantyr/DAE-libri/4549.wav
/home/zantyr/DAE-libri/14163.wav
/home/zantyr/DAE-libri/24519.wav
/home/zantyr/DAE-libri/22107.wav
/home/zantyr/DAE-libri/25772.wav
/home/zantyr/DAE-libri/26558.wav
/home/zantyr/DAE-libri/926.wav
/home/zantyr/DAE-li

/home/zantyr/DAE-libri/7598.wav
/home/zantyr/DAE-libri/23740.wav
/home/zantyr/DAE-libri/7417.wav
/home/zantyr/DAE-libri/10158.wav
/home/zantyr/DAE-libri/25311.wav
/home/zantyr/DAE-libri/226.wav
/home/zantyr/DAE-libri/17092.wav
/home/zantyr/DAE-libri/2820.wav
/home/zantyr/DAE-libri/15977.wav
/home/zantyr/DAE-libri/25776.wav
/home/zantyr/DAE-libri/11503.wav
/home/zantyr/DAE-libri/25887.wav
/home/zantyr/DAE-libri/2975.wav
/home/zantyr/DAE-libri/10564.wav
/home/zantyr/DAE-libri/15132.wav
/home/zantyr/DAE-libri/14011.wav
/home/zantyr/DAE-libri/18238.wav
/home/zantyr/DAE-libri/27515.wav
/home/zantyr/DAE-libri/15314.wav
/home/zantyr/DAE-libri/22445.wav
/home/zantyr/DAE-libri/20248.wav
/home/zantyr/DAE-libri/5732.wav
/home/zantyr/DAE-libri/19735.wav
/home/zantyr/DAE-libri/27756.wav
/home/zantyr/DAE-libri/22254.wav
/home/zantyr/DAE-libri/12674.wav
/home/zantyr/DAE-libri/1428.wav
/home/zantyr/DAE-libri/25821.wav
/home/zantyr/DAE-libri/11067.wav
/home/zantyr/DAE-libri/14463.wav
/home/zantyr/DAE-l

100%|██████████| 80/80 [00:13<00:00,  5.92it/s]


/home/zantyr/DAE-libri/26039.wav
/home/zantyr/DAE-libri/20347.wav
/home/zantyr/DAE-libri/17955.wav
/home/zantyr/DAE-libri/20782.wav
/home/zantyr/DAE-libri/2164.wav
/home/zantyr/DAE-libri/192.wav
/home/zantyr/DAE-libri/14384.wav
/home/zantyr/DAE-libri/20322.wav
/home/zantyr/DAE-libri/17586.wav
/home/zantyr/DAE-libri/264.wav
/home/zantyr/DAE-libri/9041.wav
/home/zantyr/DAE-libri/6891.wav
/home/zantyr/DAE-libri/26019.wav
/home/zantyr/DAE-libri/3948.wav
/home/zantyr/DAE-libri/14152.wav
/home/zantyr/DAE-libri/1478.wav
/home/zantyr/DAE-libri/5709.wav
/home/zantyr/DAE-libri/8414.wav
/home/zantyr/DAE-libri/28161.wav
/home/zantyr/DAE-libri/10743.wav
/home/zantyr/DAE-libri/19440.wav
/home/zantyr/DAE-libri/12955.wav
/home/zantyr/DAE-libri/21838.wav
/home/zantyr/DAE-libri/13449.wav
/home/zantyr/DAE-libri/27171.wav
/home/zantyr/DAE-libri/9903.wav
/home/zantyr/DAE-libri/8817.wav
/home/zantyr/DAE-libri/23299.wav
/home/zantyr/DAE-libri/7784.wav
/home/zantyr/DAE-libri/6060.wav
/home/zantyr/DAE-libri/20

/home/zantyr/DAE-libri/18885.wav
/home/zantyr/DAE-libri/16091.wav
/home/zantyr/DAE-libri/10055.wav
/home/zantyr/DAE-libri/11294.wav
/home/zantyr/DAE-libri/14459.wav
/home/zantyr/DAE-libri/11730.wav
/home/zantyr/DAE-libri/2079.wav
/home/zantyr/DAE-libri/11279.wav
/home/zantyr/DAE-libri/20308.wav
/home/zantyr/DAE-libri/10656.wav
/home/zantyr/DAE-libri/10181.wav
/home/zantyr/DAE-libri/26952.wav
/home/zantyr/DAE-libri/12833.wav
/home/zantyr/DAE-libri/5047.wav
/home/zantyr/DAE-libri/25928.wav
/home/zantyr/DAE-libri/16866.wav
/home/zantyr/DAE-libri/26338.wav
/home/zantyr/DAE-libri/17408.wav
/home/zantyr/DAE-libri/14684.wav
/home/zantyr/DAE-libri/27207.wav
/home/zantyr/DAE-libri/3977.wav
/home/zantyr/DAE-libri/22342.wav
/home/zantyr/DAE-libri/6391.wav
/home/zantyr/DAE-libri/24169.wav
/home/zantyr/DAE-libri/27151.wav
/home/zantyr/DAE-libri/20466.wav
/home/zantyr/DAE-libri/9461.wav
/home/zantyr/DAE-libri/8189.wav
/home/zantyr/DAE-libri/22486.wav
/home/zantyr/DAE-libri/21065.wav
/home/zantyr/DAE

100%|██████████| 80/80 [00:13<00:00,  5.94it/s]


/home/zantyr/DAE-libri/19745.wav
/home/zantyr/DAE-libri/13287.wav
/home/zantyr/DAE-libri/11263.wav
/home/zantyr/DAE-libri/13846.wav
/home/zantyr/DAE-libri/20510.wav
/home/zantyr/DAE-libri/21337.wav
/home/zantyr/DAE-libri/10930.wav
/home/zantyr/DAE-libri/8021.wav
/home/zantyr/DAE-libri/10875.wav
/home/zantyr/DAE-libri/25320.wav
/home/zantyr/DAE-libri/9821.wav
/home/zantyr/DAE-libri/17813.wav
/home/zantyr/DAE-libri/25615.wav
/home/zantyr/DAE-libri/6284.wav
/home/zantyr/DAE-libri/24826.wav
/home/zantyr/DAE-libri/9410.wav
/home/zantyr/DAE-libri/8360.wav
/home/zantyr/DAE-libri/14012.wav
/home/zantyr/DAE-libri/996.wav
/home/zantyr/DAE-libri/8226.wav
/home/zantyr/DAE-libri/13647.wav
/home/zantyr/DAE-libri/26133.wav
/home/zantyr/DAE-libri/2066.wav
/home/zantyr/DAE-libri/12528.wav
/home/zantyr/DAE-libri/10584.wav
/home/zantyr/DAE-libri/2540.wav
/home/zantyr/DAE-libri/4437.wav
/home/zantyr/DAE-libri/24173.wav
/home/zantyr/DAE-libri/17025.wav
/home/zantyr/DAE-libri/26818.wav
/home/zantyr/DAE-libr

/home/zantyr/DAE-libri/5310.wav
/home/zantyr/DAE-libri/24368.wav
/home/zantyr/DAE-libri/746.wav
/home/zantyr/DAE-libri/22463.wav
/home/zantyr/DAE-libri/15384.wav
/home/zantyr/DAE-libri/12685.wav
/home/zantyr/DAE-libri/25935.wav
/home/zantyr/DAE-libri/17170.wav
/home/zantyr/DAE-libri/20734.wav
/home/zantyr/DAE-libri/10774.wav
/home/zantyr/DAE-libri/20600.wav
/home/zantyr/DAE-libri/22060.wav
/home/zantyr/DAE-libri/12753.wav
/home/zantyr/DAE-libri/10162.wav
/home/zantyr/DAE-libri/3603.wav
/home/zantyr/DAE-libri/7066.wav
/home/zantyr/DAE-libri/21022.wav
/home/zantyr/DAE-libri/1339.wav
/home/zantyr/DAE-libri/19331.wav
/home/zantyr/DAE-libri/12079.wav
/home/zantyr/DAE-libri/12812.wav
/home/zantyr/DAE-libri/25844.wav
/home/zantyr/DAE-libri/13064.wav
/home/zantyr/DAE-libri/16706.wav
/home/zantyr/DAE-libri/6268.wav
/home/zantyr/DAE-libri/1949.wav
/home/zantyr/DAE-libri/5727.wav
/home/zantyr/DAE-libri/9937.wav
/home/zantyr/DAE-libri/1229.wav
/home/zantyr/DAE-libri/128.wav
/home/zantyr/DAE-libri/

100%|██████████| 80/80 [00:13<00:00,  5.98it/s]
